<a href="https://colab.research.google.com/github/mdjamina/model_relatives_psr_fr/blob/model_fit/src/psr_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation modules

In [ ]:
!pip install datasets

In [ ]:
!pip install -U transformers[torch]

In [ ]:
!pip install -U accelerate

# Chargement du dataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

#chargement de dataset à partir de Hugging Face
raw_datasets = load_dataset("djamina/relatives_psr_fr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/281 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/71 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/88 [00:00<?, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 281
    })
    valid: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 71
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags'],
        num_rows: 88
    })
})

# Préparation des données

## Tokenizer




In [ ]:
 #Importation de la classe AutoTokenizer de la bibliothèque 'transformers'
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("camembert-base", force_download=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

In [ ]:
# Vérification si le tokenizer est une version rapide

tokenizer.is_fast

True

### Traitement des labels

In [ ]:
# Explication de la sortie des tokens :
# - '<s>' : Token de début de séquence (utilisé pour indiquer le début d'une phrase).
# - Les caractères '▁' indiquent le début d'un nouveau mot ou d'un sous-mot, ceci est dû à l'utilisation de la tokenisation par sous-mots (subword tokenization) utilisée par le modèle CamemBERT.
# - Les sous-mots permettent de gérer efficacement les mots rares ou inconnus en les décomposant en unités plus petites.
# - Par exemple, '▁Mes' est le mot "Mes", '▁collègues' est le mot "collègues", mais 's', 'cell', 'ons' sont des sous-mots formant le mot "scellons".
# - Cette méthode améliore la capacité du modèle à généraliser à partir de morceaux de mots, surtout en présence de vocabulaire diversifié.


In [ ]:
def show(words, labels,label_names):
  line1 = ""
  line2 = ""
  for word, label in zip(words, labels):
      full_label = label_names[label]
      max_length = max(len(word), len(full_label))
      line1 += word + " " * (max_length - len(word) + 1)
      line2 += full_label + " " * (max_length - len(full_label) + 1)

  # Affichage aligné des tokens et de leurs tags
  print(line1)
  print(line2)

In [ ]:

# Extraction des features 'psr_tags' à partir du dataset
label_names = raw_datasets["train"].features["psr_tags"].feature.names
print(f'label_names : {label_names}')
print("-----------------")

words = raw_datasets["train"][15]["tokens"]
labels = raw_datasets["train"][15]["psr_tags"]

show(words, labels,label_names)



label_names : ['O', 'DET', 'APPO', 'AMBIGUE']
-----------------
Président du groupe La République en marche de l' Assemblée nationale , je mesure l' exigence de cette mission et la responsabilité collective qui  est désormais la nôtre . 
O         O  O      O  O          O  O      O  O  O         O         O O  O      O  O        O  O     O       O  O  O              O          APPO O   O         O  O     O 


In [ ]:
# cette méthode permet d'aligner les labels avec les nouveaux tokens générés par le tokenizer

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
# cette méthode permet de tokeniser le dataset (par ligne) et d'aligner les labels avec les tokens générés par le tokenizer

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["psr_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

### Tokenization

In [ ]:
# Application de la méthode 'tokenize_and_align_labels' sur le dataset 'raw_datasets'
# pour obtenir le dataset 'tokenized_datasets' qui sera utilisé pour l'entraînement du modèle

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

Map:   0%|          | 0/71 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 281
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 71
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 88
    })
})

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


# Evaluate

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Méthode pour calculer les métriques d'évaluation
# On utilise les métriques de classification NER (Named Entity Recognition)
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [
        [label_names[l] for l in label if l != -100] for label in labels
    ]
    true_predictions = [
        [label_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists to calculate metrics
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]

    results = {
        "precision": precision_score(true_labels_flat, true_predictions_flat, average='macro',zero_division=1),
        "recall": recall_score(true_labels_flat, true_predictions_flat, average='macro'),
        "f1": f1_score(true_labels_flat, true_predictions_flat, average='macro'),
        "accuracy": accuracy_score(true_labels_flat, true_predictions_flat),
    }
    return results

# Train

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    2,
            0,    0,    0,    0,    0,    0, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [ ]:
from transformers import CamembertConfig, CamembertForTokenClassification
# Définir la configuration du modèle
config = CamembertConfig(
    vocab_size=100_000,  # Taille du vocabulaire
    num_labels=4,  # Nombre de labels dans votre dataset
    hidden_size=768,  # Taille des embeddings
    num_attention_heads=12,  #le lien entre les tokens, jusqu'au 12 eme token
    num_hidden_layers=12,  # Nombre de couches
)

# Initialiser le modèle avec la configuration
model = CamembertForTokenClassification(config)

In [ ]:
model.config.num_labels

4

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "relatives_psr_fr_v21",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=False,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [ ]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.051397,0.823658,0.417863,0.365661,0.982901
2,No log,0.048867,0.706660,0.393019,0.411439,0.984801
3,No log,0.050269,0.772432,0.423804,0.450589,0.987545
4,No log,0.053608,0.679252,0.400799,0.397511,0.985223
5,No log,0.053779,0.703044,0.410832,0.424363,0.985434


In [ ]:
trainer.push_to_hub(commit_message="Training complete")

model.safetensors:   0%|          | 0.00/649M [00:00<?, ?B/s]

events.out.tfevents.1716901873.60e7450d2721.188.2:   0%|          | 0.00/7.71k [00:00<?, ?B/s]

events.out.tfevents.1716901731.60e7450d2721.188.0:   0%|          | 0.00/7.61k [00:00<?, ?B/s]

events.out.tfevents.1716901781.60e7450d2721.188.1:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

events.out.tfevents.1716901916.60e7450d2721.188.3:   0%|          | 0.00/7.71k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/djamina/relatives_psr_fr_v21/commit/17d7e4b13f5d02e14486d6a47bd174ee86707a88', commit_message='Training complete', commit_description='', oid='17d7e4b13f5d02e14486d6a47bd174ee86707a88', pr_url=None, pr_revision=None, pr_num=None)

## Inference